# Subscription Retention & CLTV Analytics (2025)

### Problem Statement
(In today’s competitive digital economy, subscription-based companies face increasing acquisition costs and revenue risk from churn. Build a system to predict which subscribers will cancel in the next billing cycle and provide actionable insights to retain high-value users.)

In today’s competitive digital economy, subscription-based companies—such as OTT platforms, SaaS products, and online service providers—face intense challenges in retaining customers and reducing churn. Customer acquisition costs are rising, and even a small increase in churn rate can cause significant revenue loss. To stay competitive, companies must predict which subscribers are at high risk of cancellation and understand why those customers are leaving. This project aims to develop a data-driven subscription retention solution that: Predicts the likelihood of a customer canceling their subscription in the next billing cycle, and Provides actionable business insights to retain valuable customers and improve customer lifetime value (CLTV).

### Business Context
(Dataset adapted from Kaggle Telco Customer Churn. Goal is to reduce churn, optimize retention campaigns and estimate CLTV impact.)
A digital services company (think Netflix, Hotstar, Spotify, or a SaaS productivity app) has collected anonymized customer subscription data — including demographic details, subscription plans, payment modes, and engagement patterns. The business team wants to: 
1. Identify high churn-risk customers before they cancel.
2. Understand key churn drivers (pricing, plan type, payment mode, engagement).
3. Optimize marketing campaigns (discounts, loyalty offers) for high-risk customers.
4. Estimate potential revenue loss if churn continues.
5. Build a live Power BI dashboard to monitor customer retention KPIs.

### Project Objectives
1. *Perform* - Exploratory Data Analysis (EDA) to identify churn trends and customer patterns. 
2. *Predictive* - Build a supervised ML model to predict churn probability per customer. 
3. *Explainable* - Use SHAP/LIME to interpret model predictions and highlight key churn drivers. 
4. *Prescriptive* - Recommend retention actions (e.g., targeted offers, upgrades). 
5. *Visual* - Create an interactive Power BI dashboard to track retention metrics and revenue impact.


## Day 1 Objectives
- Download & store Kaggle Dataset at `data/raw/telco_customer_churn.csv`.
- Initialize project structure & git.
- Document problem statement, objectives, and business context here
- Generated and loaded Sample Data
- Load main dataset and inspect: datatypes, missing values, unique values.
- Read dataset documentation and summarize feature meanings.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Loading the Data 
df = pd.read_csv('../data/raw/Telco-Customer-Churn.csv')
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [16]:
# structure & types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [17]:
# basic statistics (numerical + categorical)
df.describe(include="all")

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
count,7043,7043,7043.000000,7043,7043,7043.000000,7043,7043,7043,7043,...,7043,7043,7043,7043,7043,7043,7043,7043.000000,7043,7043
unique,7043,2,NaN,2,2,NaN,2,3,3,3,...,3,3,3,3,3,2,4,NaN,6531,2
top,7590-VHVEG,Male,NaN,No,No,NaN,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,NaN,,No
freq,1,3555,NaN,3641,4933,NaN,6361,3390,3096,3498,...,3095,3473,2810,2785,3875,4171,2365,NaN,11,5174
mean,NaN,NaN,0.162147,NaN,NaN,32.371149,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.761692,NaN,NaN
std,NaN,NaN,0.368612,NaN,NaN,24.559481,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.090047,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.250000,NaN,NaN
25%,NaN,NaN,0.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.500000,NaN,NaN
50%,NaN,NaN,0.000000,NaN,NaN,29.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.350000,NaN,NaN
75%,NaN,NaN,0.000000,NaN,NaN,55.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.850000,NaN,NaN


In [12]:
# missing values
print(df.isnull().sum().sort_values(ascending=False))

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [13]:
# unique counts (helpful to spot id-like columns)
print(df.nunique().sort_values(ascending=False))


customerID          7043
TotalCharges        6531
MonthlyCharges      1585
tenure                73
PaymentMethod          4
StreamingMovies        3
TechSupport            3
OnlineBackup           3
StreamingTV            3
DeviceProtection       3
MultipleLines          3
InternetService        3
OnlineSecurity         3
Contract               3
Partner                2
SeniorCitizen          2
gender                 2
Dependents             2
PhoneService           2
PaperlessBilling       2
Churn                  2
dtype: int64
